# Import All Libaries

In [1]:
# Imports
from tira.third_party_integrations import ensure_pyterrier_is_loaded, persist_and_normalize_run
from tira.rest_api_client import Client
ensure_pyterrier_is_loaded()
import pandas as pd
import pyterrier as pt
from tqdm import tqdm
from jnius import autoclass
import gzip
import json


import nltk
from nltk.stem import LancasterStemmer
from nltk.stem import WordNetLemmatizer

# Create a REST client to the TIRA platform for retrieving the pre-indexed data.
tira = Client()

nltk.download('punkt')
nltk.download('wordnet')

PyTerrier 0.10.0 has loaded Terrier 5.8 (built by craigm on 2023-11-01 18:05) and terrier-helper 0.0.8

No etc/terrier.properties, using terrier.default.properties for bootstrap configuration.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

# Load the Dataset and the Index

In [2]:
# The dataset: the union of the IR Anthology and the ACL Anthology
# This line creates an IRDSDataset object and registers it under the name provided as an argument.
dataset = 'antique-test-20230107-training'
pt_dataset = pt.get_dataset(f'irds:ir-benchmarks/{dataset}')
bm25 = tira.pt.from_submission('ir-benchmarks/tira-ir-starter/BM25 Re-Rank (tira-ir-starter-pyterrier)', dataset)

# A (pre-built) PyTerrier index loaded from TIRA
index = tira.pt.index('ir-lab-sose-2024/tira-ir-starter/Index (tira-ir-starter-pyterrier)', pt_dataset)

### Stemmer and Lemmatizer  (funktioniert leider nicht)

Variante 1

In [37]:
def stem(t):
    lemmatizer = WordNetLemmatizer()
    return lemmatizer.lemmatize(t)

lemmatizer = WordNetLemmatizer()


indexer = pt.IterDictIndexer("/tmp/index", overwrite=True, stemmer=lemmatizer)
index_ref = indexer.index(pt_dataset.get_corpus_iter())
index = pt.IndexFactory.of(index_ref)

bm25_stem = pt.BatchRetrieve(index, wmodel="BM25")



def stem(t):
    stemmer = LancasterStemmer()
    return stemmer.stem(t)

stemmer = LancasterStemmer()

indexer = pt.IterDictIndexer("/tmp/index", overwrite=True, stemmer=stemmer)
index_ref = indexer.index(pt_dataset.get_corpus_iter())
index = pt.IndexFactory.of(index_ref)


bm25_lem = pt.BatchRetrieve(index, wmodel="BM25")





ir-lab-sose-2024/ir-acl-anthology-20240504-training documents: 100%|██████████| 126958/126958 [00:02<00:00, 59212.78it/s]
ir-lab-sose-2024/ir-acl-anthology-20240504-training documents: 100%|██████████| 126958/126958 [00:02<00:00, 57026.63it/s]


15:32:40.883 [ForkJoinPool-13-worker-3] WARN org.terrier.structures.indexing.Indexer - Adding an empty document to the index (2020.mir_conference-2020.1) - further warnings are suppressed


ir-lab-sose-2024/ir-acl-anthology-20240504-training documents: 100%|██████████| 126958/126958 [00:18<00:00, 7013.68it/s]


15:32:46.680 [ForkJoinPool-13-worker-3] WARN org.terrier.structures.indexing.Indexer - Indexed 3 empty documents
TerrierStemmer.porter


Variante 2

In [43]:
def stem_text(text):
    stemmer = LancasterStemmer()
    return " ".join([stemmer.stem(word) for word in nltk.word_tokenize(text)])

def lemmatize_text(text):
    lemmatizer = WordNetLemmatizer()
    return " ".join([lemmatizer.lemmatize(word) for word in nltk.word_tokenize(text)])

def document_iterator(docs, transform_func):
    for doc in docs:
        doc["text"] = transform_func(doc["text"])
        yield doc


transform_func = stem_text  # oder lemmatize_text


indexer = pt.IterDictIndexer("/tmp/index", overwrite=True)
index_ref = indexer.index(document_iterator(pt_dataset.get_corpus_iter(), transform_func))

class NLTKStemmerTransformer(pt.Transformer):
    def __init__(self, stemmer):
        self.stemmer = stemmer

    def transform(self, queries):
        queries['query'] = queries['query'].apply(stem_text)
        return queries


stemmer = LancasterStemmer()
nltk_stemmer_transformer = NLTKStemmerTransformer(stemmer)


index = pt.IndexFactory.of(index_ref)


bm25_stem = nltk_stemmer_transformer >> pt.BatchRetrieve(index, wmodel="BM25")

16:24:11.578 [ForkJoinPool-15-worker-3] WARN org.terrier.structures.indexing.Indexer - Adding an empty document to the index (2020.mir_conference-2020.1) - further warnings are suppressed


ir-lab-sose-2024/ir-acl-anthology-20240504-training documents: 100%|██████████| 126958/126958 [03:52<00:00, 546.67it/s]


16:25:02.649 [ForkJoinPool-15-worker-3] WARN org.terrier.structures.indexing.Indexer - Indexed 2 empty documents


In [44]:

pt.Experiment(
    retr_systems=[bm25, bm25_stem],
    topics=pt_dataset.get_topics('text'),
    qrels=pt_dataset.get_qrels(),
    names=['BM25', 'bm25_stem'],
    eval_metrics=['recall_1000', 'ndcg_cut_5', 'ndcg_cut.10', 'recip_rank']
)



,name,recall_1000,ndcg_cut_5,ndcg_cut.10,recip_rank
0,BM25,0.825376,0.39365,0.374041,0.579877
1,bm25_stem,0.000000,0.00000,0.000000,0.000000


Leider kommt immer wieder 0.0 raus egal welche Variante ich benutze.

## Document Expansion by Query Prediction with docT5query
The basic idea is to train a model, that when given an input document, generates questions that the document might answer (or more broadly, queries for which the document might be relevant). These predicted questions (or queries) are then appended to the original documents, which are then indexed as before. The docT5query model gets its name from the use of T5 as the expansion model.

The primary advantage of this approach is that expensive neural inference is pushed to indexing time, which means that "bag of words" queries against an inverted index built on the augmented document collection are only slightly slower (due to longer documents) — but the retrieval results are much better.

First we check, if our corpus has a high recall or a lower. Our Corpus in this case is the union of the IR Anthology and the ACL Anthology. The recall may change if we use another corpus.

In [52]:
bm25 = pt.BatchRetrieve(index, wmodel="BM25")

pt.Experiment(
    retr_systems=[bm25],
    topics=pt_dataset.get_topics('text'),
    qrels=pt_dataset.get_qrels(),
    names=['BM25'],
    eval_metrics=['recall_1000']
)

,name,recall_1000
0,BM25,0.788732


As we can see, we have already a high recall. This is important for the way we implement the docT5query.
More information about the implementation in the [Tutorial](https://github.com/tira-io/teaching-ir-with-shared-tasks/blob/main/tutorials/tutorial-doc-t5-query.ipynb).

In [53]:
def doc_t5_query(dataset):
    docs = tira.get_run_output('ir-benchmarks/seanmacavaney/DocT5Query', dataset) + '/documents.jsonl.gz'
    with gzip.open(docs, 'rt') as f:
        for l in tqdm(f):
            l = json.loads(l)
            l['text'] = l['querygen']
            l['docno'] = l['doc_id']
            del l['doc_id']
            del l['querygen']
            yield l

def doc_t5_query_index(dataset):
    indexer = pt.IterDictIndexer("/tmp/index2", overwrite=True, meta={'docno': 100, 'text': 20480})
    index_ref = indexer.index(doc_t5_query(dataset))
    return pt.IndexFactory.of(index_ref)


In [54]:
indexD = doc_t5_query_index(dataset)

17:03:25.536 [ForkJoinPool-16-worker-3] WARN org.terrier.structures.indexing.Indexer - Adding an empty document to the index (3052500_1) - further warnings are suppressed


403666it [00:27, 14603.13it/s]


17:03:55.057 [ForkJoinPool-16-worker-3] WARN org.terrier.structures.indexing.Indexer - Indexed 113 empty documents


In [55]:
docs_retrieved_by_bm25 = {}

bm25_result = bm25(pt_dataset.get_topics('title'))

for _, i in tqdm(bm25_result.iterrows()):
    qid, docno = str(i['qid']), str(i['docno'])

    if qid not in docs_retrieved_by_bm25:
        docs_retrieved_by_bm25[qid] = set()
    
    docs_retrieved_by_bm25[qid].add(docno)

188633it [00:07, 23826.40it/s]


In [56]:
omit_already_retrieved_docs = lambda i: i[i.apply(lambda j: str(j['docno']) not in docs_retrieved_by_bm25[str(j['qid'])], axis=1)]
omit_already_retrieved_docs = pt.apply.generic(omit_already_retrieved_docs)

In [81]:
bm25_doct5query = pt.BatchRetrieve(indexD, wmodel="BM25")
bm25_doct5query_new = bm25_doct5query >> omit_already_retrieved_docs

DocT5Query bringt uns im normalen dataset nicht so viel, also lassen wir es weg

## Stopwords

In [3]:
bm25 = pt.BatchRetrieve(index, wmodel="BM25")

def create_index(documents, stopwords):
    indexer = pt.IterDictIndexer("/tmp/index", overwrite=True, meta={'docno': 100, 'text': 20480}, stopwords=customStopwords)
    index_ref = indexer.index(documents)
    return pt.IndexFactory.of(index_ref)

customStopwords =[
    'i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', 'your', 'yours', 
    'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', 'her', 'hers', 'herself', 
    'it', 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 
    'who', 'whom', 'this', 'that', 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 
    'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 
    'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 
    'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 
    'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 
    'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 
    'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 
    'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', 'too', 'very', 's', 't', 'can', 
    'will', 'just', 'don', 'should', 'now'
]

index = create_index(pt_dataset.get_corpus_iter(), customStopwords)

bm25_stopwords = pt.BatchRetrieve(index, wmodel="BM25")

ir-benchmarks/antique-test-20230107-training documents:   0%|          | 1931/403666 [00:02<03:54, 1710.11it/s]

12:16:56.912 [ForkJoinPool-1-worker-3] WARN org.terrier.structures.indexing.Indexer - Adding an empty document to the index (2824443_2) - further warnings are suppressed


ir-benchmarks/antique-test-20230107-training documents: 100%|██████████| 403666/403666 [00:42<00:00, 9393.07it/s] 


12:17:42.105 [ForkJoinPool-1-worker-3] WARN org.terrier.structures.indexing.Indexer - Indexed 1570 empty documents


## Query Expansion with Large Language Models

In [4]:
bm25_rm3 = bm25 >> pt.rewrite.RM3(index) >> bm25
bm25_kl = bm25 >> pt.rewrite.KLQueryExpansion(index) >> bm25

In [5]:
# llm expansions with gpt
gpt_cot = tira.pt.transform_queries('workshop-on-open-web-search/tu-dresden-03/qe-gpt3.5-cot', dataset, prefix='llm_expansion_')
gpt_sq_fs = tira.pt.transform_queries('workshop-on-open-web-search/tu-dresden-03/qe-gpt3.5-sq-fs', dataset, prefix='llm_expansion_')
gpt_sq_zs = tira.pt.transform_queries('ir-benchmarks/tu-dresden-03/qe-gpt3.5-sq-zs', dataset, prefix='llm_expansion_')

# llm expansions with llama
llama_cot = tira.pt.transform_queries('ir-benchmarks/tu-dresden-03/qe-llama-cot', dataset, prefix='llm_expansion_')
llama_sq_fs = tira.pt.transform_queries('ir-benchmarks/tu-dresden-03/qe-llama-sq-fs', dataset, prefix='llm_expansion_')
llama_sq_zs = tira.pt.transform_queries('ir-benchmarks/tu-dresden-03/qe-llama-sq-zs', dataset, prefix='llm_expansion_')

# llm expansions with flan-ul2
flan_cot = tira.pt.transform_queries('ir-benchmarks/tu-dresden-03/qe-flan-ul2-cot', dataset, prefix='llm_expansion_')
flan_sq_fs = tira.pt.transform_queries('ir-benchmarks/tu-dresden-03/qe-flan-ul2-sq-fs', dataset, prefix='llm_expansion_')
flan_sq_zs = tira.pt.transform_queries('ir-benchmarks/tu-dresden-03/qe-flan-ul2-sq-zs', dataset, prefix='llm_expansion_')

In [6]:
tokeniser = pt.autoclass("org.terrier.indexing.tokenisation.Tokeniser").getTokeniser()

def pt_tokenize(text):
    return ' '.join(tokeniser.getTokens(text))

def expand_query(topic):
  ret = ' '.join([topic['query'], topic['query'], topic['query'],  topic['query'],  topic['query'], topic['llm_expansion_query']])

  # apply the tokenization
  return pt_tokenize(ret)

# we wrap this into an pyterrier transformer
# Documentation: https://pyterrier.readthedocs.io/en/latest/apply.html
pt_expand_query = pt.apply.query(expand_query)

In [7]:
pipeline_gpt_cot = (gpt_cot >> pt_expand_query) >> bm25
pipeline_gpt_sq_fs = (gpt_sq_fs >> pt_expand_query) >> bm25
pipeline_gpt_sq_zs = (gpt_sq_zs >> pt_expand_query) >> bm25

pipeline_llama_cot = (llama_cot >> pt_expand_query) >> bm25
pipeline_llama_sq_fs = (llama_sq_fs >> pt_expand_query) >> bm25
pipeline_llama_sq_zs = (llama_sq_zs >> pt_expand_query) >> bm25

pipeline_flan_cot = (flan_cot >> pt_expand_query) >> bm25
pipeline_flan_sq_fs = (flan_sq_fs >> pt_expand_query) >> bm25
pipeline_flan_sq_zs = (flan_sq_zs >> pt_expand_query) >> bm25


### Bo1 Query Expansion

In [8]:
bo1_expansion = bm25 >> pt.rewrite.Bo1QueryExpansion(index)
# build final pipeline for retrieval
bm25_bo1 = bo1_expansion >> bm25

### Evaluation

In [9]:
pt.Experiment(
    retr_systems=[bm25, bm25_stopwords, bm25_bo1, bm25_rm3, bm25_kl, pipeline_gpt_cot, pipeline_gpt_sq_fs, pipeline_gpt_sq_zs, pipeline_llama_cot, pipeline_llama_sq_fs, pipeline_llama_sq_zs, pipeline_flan_cot, pipeline_flan_sq_fs, pipeline_flan_sq_zs],
    topics=pt_dataset.get_topics('text'),
    qrels=pt_dataset.get_qrels(),
    names=['BM25', 'BM25_stopwords','BM25_Bo1', 'BM25+RM3', 'BM25+KL', 'BM25+GPT-COT', 'BM25+GPT-SQ-FS', 'BM25+GPT-SQ-ZS', 'BM25+Llama-COT', 'BM25+Llama-SQ-FS', 'BM25+Llama-SQ-ZS', 'BM25+Flan-COT', 'BM25+Flan-SQ-FS', 'BM25+Flan-SQ-ZS'],
    eval_metrics=['recall_1000', 'ndcg_cut_5', 'ndcg_cut.10', 'recip_rank']
)

,name,recall_1000,ndcg_cut_5,ndcg_cut.10,recip_rank
0,BM25,0.788732,0.529428,0.510402,0.934803
1,BM25_stopwords,0.794154,0.536473,0.516196,0.948333
2,BM25_Bo1,0.788942,0.532035,0.505874,0.949333
3,BM25+RM3,0.779658,0.509709,0.489250,0.934375
4,BM25+KL,0.788479,0.532529,0.503925,0.949333
5,BM25+GPT-COT,0.806138,0.541391,0.506853,0.875463
6,BM25+GPT-SQ-FS,0.797366,0.541749,0.518511,0.938364
7,BM25+GPT-SQ-ZS,0.790430,0.536149,0.501605,0.921401
8,BM25+Llama-COT,0.793485,0.523303,0.495643,0.866429
9,BM25+Llama-SQ-FS,0.797566,0.547980,0.517811,0.936454


In [16]:
p1 =   pipeline_gpt_cot >> bm25_stopwords
p2 = pipeline_llama_sq_zs >> bm25_stopwords
p3 = bm25_bo1 >> bm25_stopwords
p4 = p1 >> bm25
p5 = p2 >> bm25
p6 = p3 >> bm25

p7 = bm25_stopwords >> pipeline_gpt_cot
p8 = p7 >> bm25_stopwords
p9 = p4 >> bm25_stopwords

pt.Experiment(
    retr_systems=[p1, p2, p3, p4, p5, p6, p7, p8, p9],
    topics=pt_dataset.get_topics('text'),
    qrels=pt_dataset.get_qrels(),
    names=['Prototype1', 'Prototype2', 'Prototype3', 'Prototype4', 'Prototype5', 'Prototype6', 'Prototype7', 'Prototype8', 'Prototype9'],
    eval_metrics=['recall_1000', 'ndcg_cut_5', 'ndcg_cut.10', 'recip_rank']
)

,name,recall_1000,ndcg_cut_5,ndcg_cut.10,recip_rank
0,Prototype1,0.806138,0.542748,0.511496,0.876778
1,Prototype2,0.803869,0.566553,0.532910,0.924430
2,Prototype3,0.788942,0.534332,0.511348,0.952292
3,Prototype4,0.806138,0.541391,0.506853,0.875463
4,Prototype5,0.803869,0.562714,0.529252,0.930113
5,Prototype6,0.788942,0.532035,0.505874,0.949333
6,Prototype7,0.794154,0.548624,0.516441,0.894783
7,Prototype8,0.794154,0.553862,0.520156,0.903796
8,Prototype9,0.806138,0.542748,0.511496,0.876778
